In [ ]:
import pandas as pd
import numpy as np

In [7]:
# Load the CSV
chunks = pd.read_csv('../Data/Culture.csv', chunksize=1000000, low_memory=False, sep=";")


chunk_list = []
for chunk in chunks:
    print("Processing a new chunk...")
    df1 = pd.DataFrame(chunk)
    df2 = df1[['Code_Postal']].dropna()
    chunk_list.append(df2)

# Concatenate all chunks into a single DataFrame
df = pd.concat(chunk_list, ignore_index=True)

# Display basic information about the dataset
counts = df.groupby("Code_Postal").size().reset_index(name="CultureCount")
print(counts.sort_values(by="CultureCount", ascending=False).head(10))

counts.to_csv('../DataCleaned/Culture_Cleaned.csv', index=False)

Processing a new chunk...
     Code_Postal  CultureCount
1841       33000           559
1731       31000           475
4096       67000           375
851        17000           359
4268       69003           354
2976       54000           344
4614       75006           333
4612       75004           324
4609       75001           319
3326       59000           318
